<center><h1>DSCI-552 HOMEWORK 3</h1>
<br>
<font size="3">Name: Vorapoom Thirapatarapong</font>
<br>
<font size="3">USC ID: 4397330150 Github Username: bvorapoom</font></center>

# Time Series Classification Part 1: Feature Creation/Extraction

#### import packages

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
from sklearn.preprocessing import MinMaxScaler

#### get the dataset

In [2]:
df_main = pd.DataFrame(columns = ['activities', 'dataset_number', 'df'], dtype = object)
csv_index = {
    'bending1': np.arange(1, 8, 1),
    'bending2': np.arange(1, 7, 1),
    'cycling': np.arange(1, 16, 1),
    'lying': np.arange(1, 16, 1),
    'sitting': np.arange(1, 16, 1),
    'standing': np.arange(1, 16, 1),
    'walking': np.arange(1, 16, 1)
}

In [3]:
for acti, ds_list in csv_index.items():
    for ds_no in ds_list:
        csv_path = '../data/ARem/' + acti + '/dataset' + str(ds_no) + '.csv'
        temp_df = pd.read_csv(csv_path, skiprows = 4)
        temp_df.columns = ['time' if col == '# Columns: time' else col for col in temp_df.columns]
        df_main.loc[len(df_main), :] = [acti, ds_no, temp_df]

/opt/anaconda3/lib/python3.8/site-packages/pandas/core/internals/blocks.py:993: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr_value = np.array(value)


#### the 2 sections below are to read the data into train and test set - but they are not used in these homework as the main task is to create time-domain features for all data (it will be used in HW4)

In [4]:
df_main_test = pd.DataFrame(columns = ['activities', 'dataset_number', 'df'], dtype = object)
df_main_train = pd.DataFrame(columns = ['activities', 'dataset_number', 'df'], dtype = object)

csv_index_test = {
    'bending1': np.arange(1, 3, 1),
    'bending2': np.arange(1, 3, 1),
    'cycling': np.arange(1, 4, 1),
    'lying': np.arange(1, 4, 1),
    'sitting': np.arange(1, 4, 1),
    'standing': np.arange(1, 4, 1),
    'walking': np.arange(1, 4, 1)
}

csv_index_train = {
    'bending1': np.arange(3, 8, 1),
    'bending2': np.arange(3, 7, 1),
    'cycling': np.arange(4, 16, 1),
    'lying': np.arange(4, 16, 1),
    'sitting': np.arange(4, 16, 1),
    'standing': np.arange(4, 16, 1),
    'walking': np.arange(4, 16, 1)
}

In [5]:
# loading test dataset
for acti, ds_list in csv_index_test.items():
    for ds_no in ds_list:
        csv_path = '../data/ARem/' + acti + '/dataset' + str(ds_no) + '.csv'
        temp_df = pd.read_csv(csv_path, skiprows = 4)
        temp_df.columns = ['time' if col == '# Columns: time' else col for col in temp_df.columns]
        temp_df.set_index('time', inplace = True)
        df_main_test.loc[len(df_main_test), :] = [acti, ds_no, temp_df]
        
# loading train dataset
for acti, ds_list in csv_index_train.items():
    for ds_no in ds_list:
        csv_path = '../data/ARem/' + acti + '/dataset' + str(ds_no) + '.csv'
        temp_df = pd.read_csv(csv_path, skiprows = 4)
        temp_df.columns = ['time' if col == '# Columns: time' else col for col in temp_df.columns]
        temp_df.set_index('time', inplace = True)
        df_main_train.loc[len(df_main_train), :] = [acti, ds_no, temp_df]

## (c) Feature Extraction
## Classification of time series usually needs extracting features from them. In this problem, we focus on time-domain features.

### (c)-i. Research what types of time-domain features are usually used in time series classification and list them (examples are minimum, maximum, mean, etc)

> - Central Tendency features: Mean, Median, Mode
> - Minimum, Maximum, Percentile, Decile, Quartile
> - Variability features: Standard Deviation, Variance, IQR, Range
> - Skewness and Kurtosis
> - Cross correlation between each dimension
> - parameters of ARIMA

### (c)-ii. Extract the time-domain features minimum, maximum, mean, median, standard deviation, first quartile, and third quartile for all of the 6 time series in each instance. You are free to normalize/standardize features or use them directly.

In [6]:
def gen_time_domain_features(df):
    target_cols = ['avg_rss12', 'var_rss12', 'avg_rss13', 'var_rss13', 'avg_rss23', 'var_rss23']
    assert len(target_cols) == 6, 'number of target columns to extract features is not 6 !!'
    
    temp_list = []
    for col in target_cols:
        target_df = df.loc[:, col]
        temp_min = target_df.min()
        temp_max = target_df.max()
        temp_mean = target_df.mean()
        temp_median = target_df.quantile([0.5]).values[0]
        temp_sd =  target_df.std()
        temp_1stquart = target_df.quantile([0.25]).values[0]
        temp_3rdquart = target_df.quantile([0.75]).values[0]
        temp_list.extend([temp_min, temp_max, temp_mean, temp_median, temp_sd, temp_1stquart, temp_3rdquart])
        
    return temp_list

In [7]:
td_cols = ['instance', 'min1', 'max1', 'mean1', 'median1', 'sd1', '1st_quart1', '3rd_quart1',
          'min2', 'max2', 'mean2', 'median2', 'sd2', '1st_quart2', '3rd_quart2',
          'min3', 'max3', 'mean3', 'median3', 'sd3', '1st_quart3', '3rd_quart3',
          'min4', 'max4', 'mean4', 'median4', 'sd4', '1st_quart4', '3rd_quart4',
          'min5', 'max5', 'mean5', 'median5', 'sd5', '1st_quart5', '3rd_quart5',
          'min6', 'max6', 'mean6', 'median6', 'sd6', '1st_quart6', '3rd_quart6',]
df_timedomain_features = pd.DataFrame(columns = td_cols)

In [8]:
for obs in range(len(df_main)):
    temp_df = df_main.loc[obs, 'df']
    temp_time_domain_features = gen_time_domain_features(temp_df)
    df_timedomain_features.loc[len(df_timedomain_features), :] = [obs + 1, *temp_time_domain_features]

In [9]:
df_timedomain_features.set_index('instance', inplace = True)
print(df_timedomain_features.head())

           min1   max1      mean1 median1       sd1 1st_quart1 3rd_quart1  \
instance                                                                    
1         37.25   45.0  40.624792    40.5  1.476967      39.25       42.0   
2          38.0  45.67  42.812812    42.5   1.43555       42.0      43.67   
3          35.0   47.4    43.9545   44.33  1.558835       43.0       45.0   
4          33.0  47.75  42.179813    43.5  3.670666      39.15       45.0   
5          33.0  45.75  41.678063   41.75   2.24349      41.33      42.75   

         min2  max2     mean2  ...       sd5 1st_quart5 3rd_quart5 min6  max6  \
instance                       ...                                              
1         0.0   1.3  0.358604  ...  2.188449       33.0       36.0  0.0  1.92   
2         0.0  1.22  0.372438  ...  1.995255       32.0       34.5  0.0  3.11   
3         0.0   1.7   0.42625  ...  1.999604    35.3625       36.5  0.0  1.79   
4         0.0   3.0  0.696042  ...  3.849448    30.4575

In [10]:
# normalize the time domain features using MinMaxScaler in order to interpret SD between each feature easier

scaler = MinMaxScaler()
df_timedomain_features_norm = pd.DataFrame(scaler.fit_transform(df_timedomain_features), 
                                           columns = df_timedomain_features.columns, index = df_timedomain_features.index)
                                           
print(df_timedomain_features_norm.head())
                                           

              min1      max1     mean1   median1       sd1  1st_quart1  \
instance                                                                 
1         0.776042  0.571429  0.685196  0.684211  0.188828    0.647355   
2         0.791667  0.596952  0.778537  0.768421  0.183415    0.758186   
3         0.729167  0.662857  0.827242  0.845474  0.199526    0.798489   
4         0.687500  0.676190  0.751534  0.810526  0.475507    0.643325   
5         0.687500  0.600000  0.730129  0.736842  0.288999    0.731184   

          3rd_quart1  min2      max2     mean2  ...       sd5  1st_quart5  \
instance                                        ...                         
1           0.563636   0.0  0.051755  0.076911  ...  0.072449    0.931248   
2           0.624364   0.0  0.046996  0.079939  ...  0.040816    0.902146   
3           0.672727   0.0  0.075550  0.091715  ...  0.041529    1.000000   
4           0.672727   0.0  0.152885  0.150758  ...  0.344411    0.857257   
5           0.59090

### (c)-iii. Estimate the standard deviation of each of the time-domain features you extracted from the data. Then, use Python’s bootstrapped or any other method to build a 90% bootsrap confidence interval for the standard deviation of each feature.

In [11]:
# standard deviation of each time-domain features
df_timedomain_features_norm.std(axis = 0)

min1          0.199374
max1          0.167404
mean1         0.227621
median1       0.229055
sd1           0.231590
1st_quart1    0.248004
3rd_quart1    0.186870
min2          0.000000
max2          0.301174
mean2         0.344502
median2       0.331513
sd2           0.343955
1st_quart2    0.329751
3rd_quart2    0.348119
min3          0.347819
max3          0.205963
mean3         0.225172
median3       0.218184
sd3           0.160646
1st_quart3    0.234481
3rd_quart3    0.227058
min4          0.000000
max4          0.244527
mean4         0.384436
median4       0.357996
sd4           0.315466
1st_quart4    0.388765
3rd_quart4    0.391059
min5          0.211172
max5          0.202656
mean5         0.181419
median5       0.172260
sd5           0.167811
1st_quart5    0.177416
3rd_quart5    0.178688
min6          0.106600
max6          0.213107
mean6         0.366932
median6       0.373359
sd6           0.328581
1st_quart6    0.338653
3rd_quart6    0.370030
dtype: float64

In [12]:
# 90% bootstrap confidence interval for each of the time-domain features

# changing dtype of the df to float in order to pass to bootstrapped
df_timedomain_features_norm = df_timedomain_features_norm.astype(float)

df_bootstrap_std = pd.DataFrame(columns = ['time_domain_features', 'lower_bound_bs_std', 'upper_bound_bs_std'])

# loop through each df columns and get bootstrap STD at 90% ci
for col in df_timedomain_features_norm.columns:
    bs_result = bs.bootstrap(np.array(df_timedomain_features_norm.loc[:, col]), stat_func = bs_stats.std, alpha = 0.1)
    df_bootstrap_std.loc[len(df_bootstrap_std), :] = [col, bs_result.lower_bound, bs_result.upper_bound]


In [13]:
print(df_bootstrap_std)

   time_domain_features lower_bound_bs_std upper_bound_bs_std
0                  min1            0.17319           0.225515
1                  max1           0.132749           0.206831
2                 mean1           0.202716           0.253303
3               median1           0.204308           0.255122
4                   sd1           0.207788            0.25661
5            1st_quart1           0.227062           0.269806
6            3rd_quart1           0.160146            0.21528
7                  min2                0.0                0.0
8                  max2           0.278728           0.325044
9                 mean2           0.314252           0.381121
10              median2           0.298733            0.37065
11                  sd2           0.319988           0.373711
12           1st_quart2           0.297304           0.367699
13           3rd_quart2           0.318869           0.384474
14                 min3           0.328473           0.369152
15      

### (c)-iv. Use your judgement to select the three most important time-domain features (one option may be min, mean, and max)

> Three most important time-domain features I chose are <b>Mean, Median, and Standard Deviation</b>

> Few reasons to support the selection
> - All three features show high SD and bootstrap SD after normalization, which mean there could be more information we can use in order for the classification problem.
> - Mean and Median are both chosen as they're showing the central tendency of the data. Moreover, both of which could give more information on the skewness of the data
> - SD is chosen as it gives the variability of the data.

> Few reasons to not select other features
> - Min and Max are not chosen because they are more likely to be affected by outliers, especially in this case where we did not detect and handle outliers
> - For feature 2, 4, and 6, the SD and bootstrap SD of Min feature show that there is low variation in the Min values so we're not expected to draw much information from this feature
> - 1st and 3rd quartiles are as good as SD in my opinion as their SD and bootstrap SD are high, meaning there could be information we could get in order to classify the activities. Moreover, they both represent the variability of the data.

# ISLR 3.7.4

I collect a set of data (n = 100 observations) containing a single predictor and a quantitative response. I then fit a linear regression model to the data, as well as a separate cubic regression, i.e. Y = β0 +β1X +β2X2 +β3X3 +ε.

(a) Suppose that the true relationship between X and Y is linear, i.e. Y = β0 + β1X + ε. Consider the training residual sum of squares (RSS) for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

> We'd expect the training RSS for cubic regression to be lower. Even though the cubic regression might cause overfitting, the higher flexibility of the cubic model should be able to capture noises in the training dataset better and therefore result in lower training RSS.

(b) Answer (a) using test rather than training RSS.

> We'd expect the testing RSS for linear RSS to be lower. Although the cubic regression might perform better on the training set because of its higher flexibility, that is more prone to overfitting and therefore fail to generalize to unseen data.

(c) Suppose that the true relationship between X and Y is not linear, but we don’t know how far it is from linear. Consider the training RSS for the linear regression, and also the training RSS for the cubic regression. Would we expect one to be lower than the other, would we expect them to be the same, or is there not enough information to tell? Justify your answer.

> We'd expect the traning RSS for cubic regression to be lower. Even though we don't konw how far the actual data is from linear, the higher flexibility of the cubic model would be able to fit itself to the training data better than the linear model.

(d) Answer (c) using test rather than training RSS.

> Since it is unclear how far the actual data is from linear, we cannot tell which model will perform better. In case it is closer to linear than cubic, the linear model should give lower testing RSS and vice versa.